In [6]:
dir = "/home/jcxu/interpret-sum/"
fname = 'xsum_run_record_1000.pkl'
import os
import pickle

with open(dir+fname, 'rb') as fd:
    data = pickle.load(fd)



In [8]:
from transformers import BartTokenizer
import torch
import logging
import cvxpy as cp
import numpy as np
tokenizer = BartTokenizer.from_pretrained('sshleifer/distilbart-cnn-6-6')
tokenizer_lm = BartTokenizer.from_pretrained('facebook/bart-large')
print(tokenizer.vocab_size)
print(tokenizer_lm.vocab_size)


50265
50265


In [2]:
import sys

root = logging.getLogger()
root.setLevel(logging.DEBUG)

fh = logging.FileHandler('exam_dist.log')
fh.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
formatter = logging.Formatter('<br>%(levelname)s - %(message)s')
handler.setFormatter(formatter)
fh.setFormatter(formatter)
root.addHandler(fh)

root.addHandler(handler)

In [11]:
def show_top_k(prob,prefix,name, k=5):
    prob = prob.squeeze()
    topk_v, top_idx = torch.topk(prob,k=k)
    
    index = top_idx.tolist()
    toks = [tokenizer.decode(i) for i in index]
    logging.info(f"Type: {name}")
    for i,t in enumerate(toks):
        logging.info(f"{i}: {topk_v[i].item()} {prefix}{t}")
    logging.info("\n")

kld = torch.nn.KLDivLoss(log_target=True)

def compute_kld(src_dis, tgt_dis):
    distance = kld(src_dis, tgt_dis)
    return distance.item()

def pnum(num):
    return "{:.2f}".format(num)

def compute_optimal_assignment(src_distributions, src_names, tgt_distb,tgt_name):
    src_distributions = [ x.numpy() for x in src_distributions]
    c = np.asarray(src_distributions)
    n = len(src_distributions)
    target = np.asarray(tgt_distb.numpy())

    x = cp.Variable((n))
    prob = cp.Problem(cp.Minimize( cp.sum(cp.abs(x@c - target))),
                 [cp.sum(x) == 1, x >= 0])
    prob.solve()
    logging.info(f"Optimal val: {pnum(prob.value)}")
    logging.info(f"Assignment: {[ src_names[idx] for idx in range(len(src_names))]}")
    logging.info(f"Assignment: {[ pnum(x.value[idx]) for idx in range(len(src_names))]}")
    # print(prob.value)
    # print(x.value)

In [13]:

def exam_one_example(data_pack):
    p_lm = data_pack['p_lm']
    p_imp = data_pack['p_imp']
    p_attn = data_pack['p_attn']
    p_full = data_pack['p_full']
    meta = data_pack['meta']
    document, prefix, query = meta['document'] , meta['prefix'] ,meta['query'] 
    max_vocab_size = 50264
    p_lm=p_lm.squeeze()[:max_vocab_size]
    p_imp = p_imp.squeeze()[:max_vocab_size]
    p_full = p_full.squeeze()[:max_vocab_size]
    p_attn = p_attn.squeeze()[:max_vocab_size]
    p_attn = p_attn+ 0.000000001
    p_attn = p_attn/torch.sum(p_attn)

    show_top_k(p_lm,prefix,'lm')
    show_top_k(p_imp,prefix,'imp')
    show_top_k(p_full,prefix,'full')
    show_top_k(p_attn,prefix,'attn')

    logging.info(f"Distance = KLD * 1e4")
    distributions = [p_lm, p_imp, p_attn, p_full]
    log_distributions = [torch.log(x) for x in distributions]
    dist_names = ['lm','imp','attn','full']

    for i in range(len(dist_names)):
        
        for j in range(len(dist_names)):
            if i!= j:
                _distance = kld(log_distributions[i], log_distributions[j])
                logging.info(f"From {dist_names[i]} to {dist_names[j]}\t\t { format(_distance*10000, '.2f') }")
        logging.info("\n")
    
    compute_optimal_assignment(distributions[:-1], dist_names[:-1],distributions[-1], dist_names[-1])
    logging.info("\n")

In [14]:

for _d in data:
    exam_one_example(_d)


>INFO - Type: attn
<br>INFO - 0: 0.5048747062683105 Muamba died after cardiac arrest during Bolton's FA Cup quarter-final . He .
<br>INFO - 1: 0.19427242875099182 Muamba died after cardiac arrest during Bolton's FA Cup quarter-final . He  received
<br>INFO - 2: 0.029749466106295586 Muamba died after cardiac arrest during Bolton's FA Cup quarter-final . He  inspired
<br>INFO - 3: 0.025732213631272316 Muamba died after cardiac arrest during Bolton's FA Cup quarter-final . He  fight
<br>INFO - 4: 0.023539002984762192 Muamba died after cardiac arrest during Bolton's FA Cup quarter-final . He  shocks
<br>INFO - 

<br>INFO - Distance = KLD * 1e4
<br>INFO - From lm to imp		 0.68
<br>INFO - From lm to attn		 2.85
<br>INFO - From lm to full		 0.82
<br>INFO - 

<br>INFO - From imp to lm		 0.26
<br>INFO - From imp to attn		 1.28
<br>INFO - From imp to full		 0.40
<br>INFO - 

<br>INFO - From attn to lm		 0.91
<br>INFO - From attn to imp		 2.34
<br>INFO - From attn to full		 0.76
<br>INFO - 

<br>

SolverError: Solver 'ECOS' failed. Try another solver, or solve with verbose=True for more information.

In [55]:
import cvxpy as cp
import numpy as np

In [97]:
# Define and solve the CVXPY problem.
n = 3
c = np.asarray([ [0.2,0.2,0.6,0.1],[0.2,0.1,0.2,0.6],[0.9,0.,0.,0.] ])

target = np.asarray([0.9,0.1,0.,0.])

x = cp.Variable((n))
print((x@c).shape)
print(cp.sum(x@c - target).shape)
prob = cp.Problem(cp.Minimize( cp.sum(x@c - target)),
                 [cp.sum(x) == 1, x >= 0])
prob.solve()

print(prob.value)
print(x.value)

(4,)
()
-0.09999999993197917
[1.70055203e-10 1.70055203e-10 1.00000000e+00]
